---
## Section 6: Practice Exercise

### 🎯 Challenge: Build a Content Creation Pipeline

**Objective:** Create a collaborative multi-agent system that transforms raw ideas into polished blog posts.

### Requirements

Build a **3-agent pipeline** using the collaborative (handoff) pattern:

1. **Brainstormer Agent**
   - Tool: `generate_ideas(topic: str) -> str`
   - Generates 3-5 key points about the topic
   - Returns bullet points of ideas

2. **Writer Agent**  
   - Tool: `draft_content(ideas: str) -> str`
   - Takes ideas and writes a draft blog post
   - Returns structured content with intro, body, conclusion

3. **Editor Agent**
   - Tool: `improve_writing(draft: str) -> str`
   - Polishes the draft for clarity and flow
   - Returns final polished version

### Implementation Steps

1. **Define your tools** - Create the 3 tools above (simulated output is fine)
2. **Create custom state** - What fields do you need to pass between agents?
3. **Build the 3 agents** - Each with `llm.bind_tools([...])` and tool invocation logic
4. **Construct the graph** - Linear pipeline: brainstormer → writer → editor
5. **Test it** - Try: "Create a blog post about machine learning"

### Success Criteria

✅ All 3 agents have tools bound  
✅ State passes information through the pipeline  
✅ Each agent invokes its tool  
✅ Final output is polished content  
✅ Graph compiles and runs without errors

### Bonus Challenges

- **Add a 4th agent** - Fact-checker that validates claims

### Tips

- Start by copying the collaborative pattern structure
- Keep tools simple - they can return hardcoded examples
- Test each agent individually before connecting them
- Use print statements to see the pipeline flow

**Do it yourself. Good luck!** 🚀

In [1]:
# Imports
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal, TypedDict, Annotated
import operator
import os

print("✅ All imports successful")

c:\Users\dell\Desktop\publica_jan\jenv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


✅ All imports successful


In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [4]:
from langchain.tools import tool
# Tools for collaborative agents
# Brainstormer Tool
@tool
def generate_ideas(topic: str) -> str:
    """
    Generate 3–5 key ideas about a topic.
    """
    ideas = {
        "machine learning": [
            "What machine learning is and why it matters",
            "Common real-world applications of machine learning",
            "Types of machine learning (supervised, unsupervised, reinforcement)",
            "Benefits and challenges of machine learning",
            "Future trends in machine learning"
        ]
    }

    points = ideas.get(topic.lower(), [
        f"Introduction to {topic}",
        f"Why {topic} is important",
        f"Key concepts in {topic}",
        f"Challenges in {topic}",
        f"Future of {topic}"
    ])

    return "\n".join([f"- {point}" for point in points])


# Writer Tool 
@tool
def draft_content(ideas: str) -> str:
    """
    Write a draft blog post from brainstormed ideas.
    """
    return f"""
## Introduction
Machine learning is transforming how we interact with technology. From recommendations to automation, its impact is everywhere.

## Main Content
{ideas}

Machine learning systems learn from data and improve over time without being explicitly programmed. Different learning types serve different purposes across industries.

## Conclusion
As data continues to grow, machine learning will remain a critical technology shaping the future of innovation.
""".strip()


# ---------- Editor Tool ----------
@tool
def improve_writing(draft: str) -> str:
    """
    Improve clarity, flow, and structure of a draft.
    """
    return f"""
✨ Polished Blog Post ✨

{draft}

This article has been refined for better flow, clearer explanations, and smoother transitions while keeping the original meaning intact.
""".strip()


print("✅ Content-creation collaborative tools ready")


✅ Content-creation collaborative tools ready


In [5]:
from typing import TypedDict

class BlogState(TypedDict):
    topic: str
    ideas: str
    draft: str
    final: str


In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# -------- Brainstormer Agent --------
def brainstormer_agent(state: BlogState):
    ideas = generate_ideas.invoke(state["topic"])
    print("🧠 Brainstormer Output:\n", ideas)
    return {"ideas": ideas}


# -------- Writer Agent --------
def writer_agent(state: BlogState):
    draft = draft_content.invoke(state["ideas"])
    print("✍️ Writer Output:\n", draft)
    return {"draft": draft}


# -------- Editor Agent --------
def editor_agent(state: BlogState):
    final = improve_writing.invoke(state["draft"])
    print("📝 Editor Output:\n", final)
    return {"final": final}


In [7]:
from langgraph.graph import StateGraph, END

graph = StateGraph(BlogState)

graph.add_node("brainstormer", brainstormer_agent)
graph.add_node("writer", writer_agent)
graph.add_node("editor", editor_agent)

graph.set_entry_point("brainstormer")

graph.add_edge("brainstormer", "writer")
graph.add_edge("writer", "editor")
graph.add_edge("editor", END)

blog_pipeline = graph.compile()

print("✅ Graph compiled successfully")


✅ Graph compiled successfully


In [8]:
result = blog_pipeline.invoke(
    {"topic": "machine learning"}
)

print("\n🎉 FINAL OUTPUT:\n")
print(result["final"])


🧠 Brainstormer Output:
 - What machine learning is and why it matters
- Common real-world applications of machine learning
- Types of machine learning (supervised, unsupervised, reinforcement)
- Benefits and challenges of machine learning
- Future trends in machine learning
✍️ Writer Output:
 ## Introduction
Machine learning is transforming how we interact with technology. From recommendations to automation, its impact is everywhere.

## Main Content
- What machine learning is and why it matters
- Common real-world applications of machine learning
- Types of machine learning (supervised, unsupervised, reinforcement)
- Benefits and challenges of machine learning
- Future trends in machine learning

Machine learning systems learn from data and improve over time without being explicitly programmed. Different learning types serve different purposes across industries.

## Conclusion
As data continues to grow, machine learning will remain a critical technology shaping the future of innovatio